In [161]:
# Sample data

import numpy as np

FEE_HAIRCUT = 1 - 0.003
SWAP_CURVE_Y = np.arange(0.01, 1.0, 0.1 )
CHAIN_LIQUIDITY_TOK_1 = [2.0, 2.0, 2.0, 2.0]
CHAIN_LIQUIDITY_TOK_2 = [1.0, 1.0, 1.0, 1.0]
MAX_RATIO_OUTSWAP_TO_LIQUIDITY = 0.9
MAX_RATIO_INSWAP_TO_LIQUIDITY = 9.0

sample_swap_init_Y_str = "\tSwap curve Y: "
sample_chain_tok_1_str = "\tChain liquidity token 1:"
sample_chain_tok_2_str = "\tChain liquidity token 2:"
sample_fee_haircut_str = "\tFee haircut:"
print(f"Sample data \n"
      + sample_swap_init_Y_str + f"{SWAP_CURVE_Y} \n"
      + sample_chain_tok_1_str + f"{CHAIN_LIQUIDITY_TOK_1}\n"
      + sample_chain_tok_2_str + f"{CHAIN_LIQUIDITY_TOK_2}\n"
      + sample_fee_haircut_str + f"{FEE_HAIRCUT:6.4f}\n\n" )

Sample data 
	Swap curve Y: [0.01 0.11 0.21 0.31 0.41 0.51 0.61 0.71 0.81 0.91] 
	Chain liquidity token 1:[2.0, 2.0, 2.0, 2.0]
	Chain liquidity token 2:[1.0, 1.0, 1.0, 1.0]
	Fee haircut:0.9970




In [162]:
# Define forward and backward swap calculators
from typing import Union

def forward_swap(x:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(x, float):
        assert x > 0
    elif isinstance(x, np.ndarray):
        assert np.all(x) > 0
    assert haircut > 0
    return haircut * x /(1 + haircut * x)

def backward_swap(y:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(y, float):
        assert y > 0.0
        assert y < 1.0
    elif isinstance(y, np.ndarray):
        assert np.all(y) > 0
        assert np.all(y < 1.0)
    assert haircut > 0
    return haircut * y /(1 - haircut * y)



In [163]:
# Define swap curves
#    Note: curves define the swap space, so haircut does not apply

x = backward_swap(SWAP_CURVE_Y, haircut=1.0)
y = forward_swap(x, haircut=1.0)

# Check if forward_swap equals inverse of backward_swap
inverse_test = np.allclose(y, SWAP_CURVE_Y)
print(f"Swap curves defined. \nForward swap is inverse of Backward swap: {inverse_test}")


Swap curves defined. 
Forward swap is inverse of Backward swap: True


In [164]:
print(x)
print(y)

[1.01010101e-02 1.23595506e-01 2.65822785e-01 4.49275362e-01
 6.94915254e-01 1.04081633e+00 1.56410256e+00 2.44827586e+00
 4.26315789e+00 1.01111111e+01]
[0.01 0.11 0.21 0.31 0.41 0.51 0.61 0.71 0.81 0.91]


In [179]:
# Cyclic transaction calculations

def generate_exchange_data(in_leg_liq: np.ndarray, out_leg_liq: np.ndarray) -> list[dict]:
    assert len(in_leg_liq) == len(out_leg_liq)
    exchange_data = []
    for i in range(len(in_leg_liq)):
        exch = { "exch_number": i+1,
                 "liq_in": in_leg_liq[i],
                 "liq_out": out_leg_liq[i],
                 "eta": 0,
                 "upper_bound": 0,
                 "delta_in": 0,
                 "delta_out": 0,}
        exchange_data.append(exch)
    return exchange_data

def generate_upper_bounds(exch_data: list[dict], x_curve: np.ndarray, y_curve: np.ndarray) -> float:

    # Initialize: delta_2 upper bound is Exchange 1 outgoing liquidity
    #             eta is delta_2 upper bound
    i = 0
    exch_data[i]['upper_bound'] = exch_data[i]['liq_out'] * MAX_RATIO_OUTSWAP_TO_LIQUIDITY
    exch_data[i]['eta'] = exch_data[i]['upper_bound']

    #print(f"Exchange number {exch_data[i]["exch_number"]}" )
    #print(f"\t Outgoing leg liquidity: {exch_data[i]['liq_out']}")
    #print(f"\t Upper bound outgoing leg: {exch_data[i]['upper_bound']}")
    #print(f"\t Eta: {exch_data[i]['eta']}")

    print(f"Exchange data: {exch_data[i]}")

    # Eta and bound loop
    for i in range(1,len(exch_data)):
        exch_prior = exch_data[i-1]["exch_number"]
        exch_current = exch_data[i]['exch_number']
        exch_prior_out_liq = exch_data[i-1]['liq_out']
        exch_current_in_liq = exch_data[i]['liq_in']
        exch_current_out_liq = exch_data[i]['liq_out']

    #    print(f"Exchange number {exch_current}" )
    #    print(f"\t Prior exchange {exch_prior}" )
    #    print(f"\t Prior exchange outgoing leg liquidity: {exch_prior_out_liq}")
    #    print(f"\t Current exchange {exch_current}" )
    #    print(f"\t Current exchange incoming leg liquidity: {exch_current_in_liq}")

        # a(i, i-1)/a(i,i+1)
        ratio_out_div_by_in = exch_prior_out_liq / exch_current_in_liq
        xi_indx = np.searchsorted(x_curve, ratio_out_div_by_in, side='right')

     #   print(f"\t Ratio {ratio_out_div_by_in}")
     #   print(f"\t X value above {x_curve[xi_indx]}")
     #   print(f"\t X value below {x_curve[xi_indx-1]}")

        eta = float(y_curve[xi_indx]) * exch_data[i-1]['eta']
        exch_data[i]['upper_bound'] = eta *  exch_current_out_liq
        exch_data[i]['eta'] = eta

        print(f"Exchange data: {exch_data[i]}")

    return exch_data[len(exch_data)-1]['upper_bound']



In [180]:
exch_cycle = generate_exchange_data(CHAIN_LIQUIDITY_TOK_1, CHAIN_LIQUIDITY_TOK_2)

In [181]:
delta_1_prime = generate_upper_bounds(exch_data=exch_cycle, x_curve=x, y_curve=y)

Exchange data: {'exch_number': 1, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.9, 'upper_bound': 0.9, 'delta_in': 0, 'delta_out': 0}
Exchange data: {'exch_number': 2, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.36900000000000005, 'upper_bound': 0.36900000000000005, 'delta_in': 0, 'delta_out': 0}
Exchange data: {'exch_number': 3, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.15129000000000004, 'upper_bound': 0.15129000000000004, 'delta_in': 0, 'delta_out': 0}
Exchange data: {'exch_number': 4, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.06202890000000002, 'upper_bound': 0.06202890000000002, 'delta_in': 0, 'delta_out': 0}


In [182]:
# Compute swaps backward from last exchange upper bound

i = len(exch_cycle)-1
exch_cycle[i]['delta_out'] = delta_1_prime

for i in range(len(exch_cycle)-1, -1, -1):
    print(i)
    liq_out = exch_cycle[i]['liq_out']
    liq_in = exch_cycle[i]['liq_in']
    delta_out = exch_cycle[i]['delta_out']
    delta_in = liq_in * backward_swap(delta_out/liq_out)
    exch_cycle[i]['delta_in'] = delta_in
    if i > 0:
        exch_cycle[i-1]['delta_out'] = exch_cycle[i]['delta_in']

    print(f"Exchange data: {exch_cycle[i]}")


3
Exchange data: {'exch_number': 4, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.06202890000000002, 'upper_bound': 0.06202890000000002, 'delta_in': 0.1318389160723359, 'delta_out': 0.06202890000000002}
2
Exchange data: {'exch_number': 3, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.15129000000000004, 'upper_bound': 0.15129000000000004, 'delta_in': 0.30267088920131197, 'delta_out': 0.1318389160723359}
1
Exchange data: {'exch_number': 2, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.36900000000000005, 'upper_bound': 0.36900000000000005, 'delta_in': 0.8643564382130018, 'delta_out': 0.30267088920131197}
0
Exchange data: {'exch_number': 1, 'liq_in': 2.0, 'liq_out': 1.0, 'eta': 0.9, 'upper_bound': 0.9, 'delta_in': 12.467945175324177, 'delta_out': 0.8643564382130018}
